# Assignment 1: Transformations


TEAM-ID:   20
TEAM-NAME:   formula-ai
YOUR-ID:   2018122003
YOUR-NAME:    Krishna Kumar Maram

(Although you work in groups, both the students have to submit to Moodle, hence there's name field above)

# Instructions
- Please check Moodle for "TEAM-ID" and "TEAM-NAME" fields above. Some of your names have been edited because of redundancy/simplicity. Instructions for submitting the assignment through GitHub Classrooms/Moodle has been uploaded on Moodle. Any clarifications will be made there itself.
- Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment. See `./misc/installation` for detailed step-by-step instructions about the installation setup.
- Both the team members must submit the zip file.
- For this assignment, you will be using Open3D extensively. Refer to [Open3D Documentation](http://www.open3d.org/docs/release/): you can use the in-built methods and **unless explicitly mentioned**, don't need to code from scratch for this assignment. Make sure your code is modular since you will be reusing them for future assignments.
- Take a look at the entire assignment. The descriptive questions at the end might have a clue if you are stuck somewhere.
- Answer the descriptive questions in your own words with context & clarity. Do not just copy-paste from some Wikipedia page. You will be evaluated accordingly.
- Please call the visualization functions only when they are asked. They are asked explicitly at the end of every section.
- You could split the Jupyter Notebook cells where `TODO` is written, but please try to avoid splitting/changing the structure of other cells.

In [1]:
import open3d as o3d
import copy
import numpy as np
from scipy.linalg import logm
from scipy.spatial.transform import Rotation as R
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
%matplotlib inline
import time

# 1. Getting started with Open3D
## 1.1 Converting RGBD image into Point Cloud


In your robotics journey, it is common to be given just the depth images along with camera parameters in a generic dataset and you'd want to build a 3D data representation out of it, for example, a point cloud. You will understand the math behind these concepts in detail during Vision classes, for now, you can use the in-built functions as a black box.
- Below are the given RGB and D images from SUN RGB-D dataset ([S.Song, CVPR 2015](https://rgbd.cs.princeton.edu/)). 
![low-res-RGBD.resized.png](./misc/low-res-RGBD.png)

- Read these two images `color.jpg` and `depth.png` given in current folder using Open3D. Convert it to a point cloud using the default camera parameters (`o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault`).
Then,
- Create a "world" frame $A$ and combine this (just use $+$ operator) with the above point cloud and save it as `scene.pcd`. Put it aside for now.
- Write a simple function `one_one` to visualize `scene.pcd`.

In [4]:
##############################################################################
# TODO: Do tasks described in 1.1                                            #
##############################################################################
# Read both the images and create rgbd image format
color_raw = o3d.io.read_image(
        "color.jpg")
depth_raw = o3d.io.read_image(
        "depth.png")
rgbd_image = o3d.geometry.RGBDImage.create_from_sun_format(
        color_raw, depth_raw)
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image,
    o3d.camera.PinholeCameraIntrinsic(
        o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault))
# Flip it, otherwise the pointcloud will be upside down
pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
        size=0.6, origin=[0, 0, 0])
pcd_frame = mesh_frame.sample_points_poisson_disk(number_of_points=2000)
o3d.io.write_point_cloud("scene.pcd", pcd+pcd_frame)

def one_one(pcd):
    o3d.visualization.draw_geometries([pcd_frame+pcd])

##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################a

### Question for 1.1

- In the next code cell, call the function `one_one` here showing `scene.pcd`.

In [3]:
#uncomment the following and add input parameters if any
pcd_scene = o3d.io.read_point_cloud('scene.pcd')
one_one(pcd_scene)

# 2. Rotations, Euler angles and Gimbal Lock

## 2.1 Rotating an object

The objective here is to roughly simulate an object moving on a ground.

- Generate a cube at some point on the ground and create another frame $B$ at the center of this object. Combine these both as a single point cloud `cube.pcd`. (You can pick a point on the ground by using the `get_picked_points` method of the class `open3d.visualization.VisualizerWithEditing`.)
- Now read both the point clouds `scene.pcd` and `cube.pcd` in a script. Whatever tasks you do below are on the object `cube.pcd` (along with the axes $B$) with `scene.pcd` in the background (static).
- Given a sequence of **ZYX Euler** angles $[30^{\circ}, 90^{\circ}, 45^{\circ}]$, generate the rotation. In our case, our object (with its respective axis) undergoes rotation with the background being fixed (with its respective axis).
- Note: Throughout this assignment, we will be using the standard **ZYX** Euler angle convention.
- Write a function `two_one` to show the above by **animation** (cube rotating along each axis one by one).
    - *Hint: Use Open3D's non-blocking visualization and discretize the rotation to simulate the animation. For example, if you want to rotate by $30^{\circ}$ around a particular axis, do in increments of $5^{\circ}$ 6 times to make it look like an animation.*

In [32]:
##############################################################################
# TODO: Do tasks described in 2.1                                            #
##############################################################################
color = [194/255, 122/255, 192/255]
edge = 0.4
center = [-edge/2 for i in range(3)]
frame = 0.6
samples = 5000

# Create the cube
mesh_cube = o3d.geometry.TriangleMesh.create_box(width=edge, height=edge, depth=edge)
mesh_cube.paint_uniform_color(color)

mesh_frame_cube = o3d.geometry.TriangleMesh.create_coordinate_frame(size=frame, origin=center)
framed_cube = (mesh_cube + mesh_frame_cube).sample_points_poisson_disk(number_of_points = samples, init_factor=5)

o3d.io.write_point_cloud('cube.pcd', framed_cube)

scene = o3d.io.read_point_cloud('scene.pcd')
cube = o3d.io.read_point_cloud('cube.pcd')
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################


In [33]:
def update(vis, rotation, center, cube):
    cube.rotate(rotation, center = center)
    vis.update_geometry(cube)
    vis.poll_events()
    vis.update_renderer()
    time.sleep(0.1)

def two_one(scene,cube):
#     create visualizer object
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(scene)
    vis.add_geometry(cube)
    
    val = (np.pi)/36
    
    ROA = cube.get_rotation_matrix_from_zyx((val,0,0))
    for i in range(0,6):
        update(vis,ROA,center,cube)
    
    ROA = cube.get_rotation_matrix_from_zyx((val*6,0,0))
    ROB = cube.get_rotation_matrix_from_zyx((val*6,val,0))
    
    RAB = ROB@(ROA.T)
    for i in range(0,18):
        update(vis,RAB,center,cube)
    
    ROB = cube.get_rotation_matrix_from_zyx((val*6, val*18, 0))
    ROC = cube.get_rotation_matrix_from_zyx((val*6, val*18, val))
    
    RBC = ROC@(ROB.T)
    for i in range(0,9):
        update(vis,RBC,center,cube)
        
    vis.destroy_window()

### Question for 2.1

- In the next code cell, call the function `two_one` here showing the animation described in section 2.1.

In [34]:
#uncomment the following and add input parameters if any
cube_corner = np.random.rand(3,1)
scene = o3d.io.read_point_cloud('scene.pcd')
cube = o3d.io.read_point_cloud('cube.pcd')
cube.translate(cube_corner)
two_one(scene,cube)

## 2.2 Euler angle & Gimbal lock

Code the following yourself from scratch (Refer Craig book - Section: $Z-Y-X$ Euler angles - same conventions/notations followed).

- Case 1: Given the rotation matrix $M_{given}$ below, extract Euler angles $\alpha , \beta ,\gamma$. Convert it back to the rotation matrix $M_{recovered}$ from Euler angles.

    $$M(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0.26200263 & -0.19674724 & 0.944799 \\0.21984631 & 0.96542533 & 0.14007684 \\
    -0.93969262 & 0.17101007 & 0.29619813\end{array}\right] $$

    After coding it from scratch, check your calculations using `scipy.spatial.transform.Rotation`. (Mandatory)

- Case 2: Given the rotation matrix $N_{given}$, extract Euler angles, and convert back $N_{recovered}$.

    $$N(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0 & -0.173648178 &  0.984807753 \\0 & 0.984807753 & 0.173648178 \\
    -1 & 0 & 0\end{array}\right] $$

    Again use `scipy` and check its output. If `scipy` is showing any warnings on any of the above cases, explain it in "**Questions for 2.2**" (last question). Write code in the next cell.
    
- (Optional) Case 3: Do the above two for quaternion using scipy functions, i.e. given the rotation matrix, extract quaternion and convert back.

In [35]:
##############################################################################
# DON'T EDIT
M_given =  np.array([[0.26200263, -0.19674724, 0.944799],
                     [0.21984631, 0.96542533, 0.14007684],
                     [-0.93969262, 0.17101007, 0.29619813]])

N_given = np.array([[0,-0.173648178,0.984807753],
                    [0, 0.984807753, 0.173648178],
                    [-1, 0, 0]])

In [36]:
# TODO: Do tasks described in 2.2                                            #
##############################################################################
# Replace "pass" statement with your code
def euler_from_rotation(R):
    beta = np.arctan2(-R[2,0],np.sqrt(R[0,0]**2+R[1,0]**2))
    alpha = np.arctan2(R[1,0]/np.cos(beta),R[0,0]/np.cos(beta))
    gamma = np.arctan2(R[2,1]/np.cos(beta),R[2,2]/np.cos(beta))
    return (alpha,beta,gamma)
def rotation_from_euler(alpha,beta,gamma):
    R = np.zeros((3,3))
    cos = np.cos(np.array([alpha,beta,gamma]))
    sin = np.sin(np.array([alpha,beta,gamma]))
    R[0,0] = cos[0]*cos[1]
    R[0,1] = cos[0]*sin[1]*sin[2] - sin[0]*cos[2]
    R[0,2] = cos[0]*sin[1]*cos[2] + sin[0]*sin[2]
    
    R[1,0] = sin[0]*cos[1]
    R[1,1] = sin[0]*sin[1]*sin[2] + cos[0]*cos[2]
    R[1,2] = sin[0]*sin[1]*cos[2] - cos[0]*sin[2]
    
    R[2,0] = -sin[1]
    R[2,1] = cos[1]*sin[2]
    R[2,2] = cos[1]*cos[2]
    return R
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

In [37]:
angles_M = euler_from_rotation(M_given)
angles_scipy_M = R.from_matrix(M_given).as_euler('ZYX')
M_rec = rotation_from_euler(*angles_M)
print(M_given)
print(M_rec)

[[ 0.26200263 -0.19674724  0.944799  ]
 [ 0.21984631  0.96542533  0.14007684]
 [-0.93969262  0.17101007  0.29619813]]
[[ 0.26200263 -0.19674724  0.944799  ]
 [ 0.21984631  0.96542534  0.14007684]
 [-0.93969262  0.17101007  0.29619813]]


In [38]:
angles_N = euler_from_rotation(N_given)
angles_scipy_N = R.from_matrix(N_given).as_euler('ZYX')
N_rec = rotation_from_euler(*angles_N)
print(N_given)
print(N_rec)

[[ 0.         -0.17364818  0.98480775]
 [ 0.          0.98480775  0.17364818]
 [-1.          0.          0.        ]]
[[ 6.123234e-17  0.000000e+00  1.000000e+00]
 [ 0.000000e+00  1.000000e+00  0.000000e+00]
 [-1.000000e+00  0.000000e+00  6.123234e-17]]


Here we see that in case of quaternions we are able to recover N

In [39]:
angles_Q = R.from_matrix(N_given).as_quat()
Q_rec  = R.from_quat(angles_Q).as_matrix()
print(Q_rec)

[[ 0.         -0.17364818  0.98480775]
 [ 0.          0.98480775  0.17364818]
 [-1.          0.          0.        ]]


## Questions for 2.2
- Have you used `np.arctan` or an any equivalent `atan` function above? Why or why not?   
    * Ans: No since the purpose of using two arguments instead of one is to gather information on the signs of the inputs in order to return the appropriate quadrant of the computed angle, which is not possible for the single-argument Atan

### For Case 1 above,
- What Euler angles  $\alpha , \beta ,\gamma$ did you get? Replace `my_array_case1` with your array.

In [40]:
# Uncomment and replace my_array_case1 with your array.
print("My Euler angles for case 1 are" + str(np.array(angles_M)))

My Euler angles for case 1 are[0.6981317  1.22173048 0.52359878]


  - Were you able to recover back your rotation matrix when you converted it from Euler angles? Why/why not? Replace `M_given` and `M_recovered` with your matrices below and explain "why/why not" after this code snippet.
  Answer: Yes  I was able to recover both the rotation matrices. Both the matrices are orthogonal matrices. And all orthogonal matrices are valid rotation matrices.

In [41]:
# Uncomment and Replace M_given and M_recovered with your matrices below.
error = np.linalg.norm(logm(M_given @ M_rec.T))
print("For case 1, it is " + str(error<0.0001) + " I could recover the original matrix.")

For case 1, it is True I could recover the original matrix.


- Why/why not? Based on your observations here, is there any problem with Euler angle representation for Case 1? If yes, what is it?

    - Ans:   In this case there was no issue with recovery of the matrix. 

### Repeat the above for Case 2.

In [42]:
# Uncomment and Replace N_given and N_recovered with your matrices below.
print("My Euler angles for case 2 are" + str(np.array(angles_N)))
error = np.linalg.norm(logm(N_given @ N_rec.T))
print("For case 2, it is " + str(error<0.0001) + " I could recover the original matrix.")

My Euler angles for case 2 are[0.         1.57079633 0.        ]
For case 2, it is False I could recover the original matrix.


* Why/why not? Based on your observations here, is there any problem with Euler angle representation for Case 2? If yes, what is it?

    * Ans: These formulas fail if r12=r13=0,or r12=r23=0,or r23=r13= 0, because they lead to indeterminations of the form 0/0.They correspond precisely to the cases in which the input rotation matrix represents a pure rotation about the x,y,or z axis, respectively
    
    
* Explain any more problems with Euler angle representation. Explain what you understand by Gimbal lock (concisely in your own words). You could revisit this question in the section 2.4.
    * Ans: Euler angles suffer from the problem of gimbal lock, where the representation loses a degree of freedom and it is not possible to determine the first and third angles uniquely. Scipy sets the angle to zero

- When you used `scipy.spatial.transform.Rotation` for the above 2 cases,
    - Have you used `zyx` above in `r.as_euler('')` argument? Why or why not? Explain the difference between extrinsic and instrinsic rotations with equivalent technical names from Craig book?
        * Ans: Yes we have used. Although the math would be same, there is slight difference in xyz notation and zyx notation. Extrinsic(zyx) means the original coordinate system remains motionless as the rotations go.Intrinsic(xyz) means the rotation is always based on the rotating coordinate system which is solidary with the moving object.
    - Has `scipy` shown any warnings on any of the above cases? If yes, explain it.
        * Ans: The warnings were regarding gimbal lock where the representation loses a degree of freedom and it is not possible to determine the first and third angles uniquely. 
    - (Optional) For Case 3 above (quaternion) which you did using scipy, did you observe any problem with quaternion? Depending on your observations, which is better? Quaternion or Euler angles? And why?
         * Ans: Quaternions are better since they dont face an issue of Gimbal lock. They  areeasier to compute with (for the computer, not for humans) and more efficient


## 2.3 Rotation matrix as an Operator
This question will help you in your understanding of [Rotator-transform (Vector-frame) equivalence](https://www.notion.so/saishubodh/Lecture-2-Transformations-11d69d8cef2d4cd195a98fa7d33224e1#f90ece4f5e374743bfed47e46a83ecfe).

![image.png](./misc/xyz-frame.png)
Consider the frame $XYZ$ in the above image. Say you have a vector $x_1=[0,\sqrt{3},0]$. Now you want to rotate it such that you end up at $x_2=[1,1,1]$ through a sequence of Euler angle rotations. Your goal is to find out those $\alpha, \beta \: \& \: \gamma$ ($ZYX$). We will follow this order whenever we refer to it below.

First, properly understand the so-called "Rotator-transform equivalence" to figure out what are the terms of rotation matrix. Then, put the math on paper and you will end up with a set of non-linear equations. Write the set of linear equations in LaTeX here:

$$ \textbf{Ans: Your set of equations here} $$
$$ \sqrt{3}\sin{\gamma} \cos{\beta} - 1 = 0 $$
$$ \sqrt{3}\cos{\alpha}\cos{\gamma} + \sqrt{3}\sin{\alpha}\sin{\beta}\sin{\gamma} - 1 = 0 $$
$$ -\sqrt{3}\sin{\alpha}\cos{\gamma} + \sqrt{3}\cos{\alpha}\sin{\beta}\sin{\gamma} - 1 = 0 $$

Solve these equations using `fsolve` from `scipy.optimize` as follows: (Come back and answer the following questions after coding it in the next block)
- `case1`: First, solve it with an initialization of (0,0,0). Check if your answer is correct using `np.isclose`.
    * What Euler angles did you get? Answer in $\alpha, \beta \: \& \: \gamma$ format:
        * Ans: *$a_1, b_1, c_1$ = array([ -0.32906483, -93.69048524,   0.74815098])* 
- `case2`: Now, forget about the solver for a moment: Can you visualize and think of sequence of rotations one by one to reach the final position (which is different than previous set of rotations)? Now, validate your answer by giving (your answer $\pm 5$) as initialization.
    * What Euler angles did you get? Answer in $\alpha, \beta \: \& \: \gamma$ format:
        * Ans: *$a_2, b_2, c_2$ = array([5.49778714, 0.        , 0.61547971])*

In [43]:
##############################################################################
# DON'T EDIT
x_1 = np.array([0,np.sqrt(3),0])
x_2 = np.array([1,1,1])

In [44]:
##############################################################################
# TODO: Do tasks described in 2.3                                            #
##############################################################################
# Replace "pass" statement with your code
a1 = -1
b1 = -1
c1 = -1
def optim(x):
    alpha = np.sqrt(3) * np.sin(x[2]) * np.cos(x[1]) - 1
    beta = np.sqrt(3) * np.cos(x[0]) * np.cos(x[2]) + np.sqrt(3) * np.sin(x[0]) * np.sin(x[1]) * np.sin(x[2]) - 1
    gamma = -1 * np.sqrt(3) * np.sin(x[0]) * np.cos(x[2]) + np.sqrt(3) * np.cos(x[0]) * np.sin(x[1]) * np.sin(x[2]) -1
    return [alpha,beta,gamma]

root1 = fsolve(optim, [0 ,0 ,0 ])
a1, b1, c1 = root1
print("Case 1")
print(np.isclose(optim(root1), [0, 0, 0]))

print("Case 2")
root2 = fsolve(optim, [7*np.pi/4 ,0 ,np.arcsin(1/np.sqrt(3)) ])
a2, b2, c2 = root2
print(np.isclose(optim(root2), [0, 0, 0]))

##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

Case 1
[ True  True  True]
Case 2
[ True  True  True]


In [45]:
# From Section 2.2, use the function which takes Euler angles and gives Rotation matrix as output.
# Uncomment and replace `r_mat` with the name of the function. (Do NOT edit anything else)

x_2_obtained_case1 = rotation_from_euler(a1,b1,c1) @ x_1.T #TODO: replace r_mat
x_2_obtained_case2 = rotation_from_euler(a2,b2,c2) @ x_1.T #TODO: replace r_mat
test = True #TODO: Set this as True

In [46]:
# DON'T EDIT
if test == True:
    case1_test = np.isclose(x_2_obtained_case1,  np.array([1.0,1.0,1.0]))
    case2_test = np.isclose(x_2_obtained_case2,  np.array([1.0,1.0,1.0]))
    print("For case 1, it is " + str(bool(case1_test[0] and case1_test[1] and case1_test[2])) + 
          " that I could end up at (1,1,1) after rotation.")
    print("For case 2, it is " + str(bool(case2_test[0] and case2_test[1] and case2_test[2])) + 
          " that I could end up at (1,1,1) after rotation.")

For case 1, it is True that I could end up at (1,1,1) after rotation.
For case 2, it is True that I could end up at (1,1,1) after rotation.


## 2.4 Gimbal Lock visualization (Optional)

A nice visualization video for gimbal lock is [this](https://www.youtube.com/watch?v=zc8b2Jo7mno). You are about to animate a similar visualization demonstrating gimbal lock 😃.

![image.png](./misc/xyz-frame.png)

- Write a function `two_four` for the visualization of gimbal lock. Follow the below steps to get the intuition going. Use Open3D for the following.
    - Say our frame's initial position is as the above image. Now the final goal at the end of rotation is to get the $Y$ axis pointing in the direction of the vector $(x,y,z)$ that you currently see in the above image. This point is fixed in space and is NOT moving as we rotate our axis.
    - For creating that point, you could use a small sphere using `open3d.geometry.create_mesh_sphere`. You already know how to create an axis by now.
    - Following our $ZYX$ convention, first rotate your frame about $Z$ axis by an angle, say $-35^{\circ}$. Then rotate about $Y$ axis by an angle ${\beta}$ and then about $X$ by say $55^{\circ}$.
        - Are there any specific angle(s) $\beta$ using which you will **never** reach our point $(x,y,z)$ ?
            - Clue: We are specifically talking about gimbal lock here and notice the word "never".
        - Under this (these) specific angle(s) of $\beta$ & different combinations of $\alpha$ and $\gamma$, make an animation and clearly show why your $Y$ axis is unable to align in the direction of that vector $(x,y,z)$ using the animation.

            If you are unsure to simulate the animation, you could do it as follows:

            - You could first fix some $\alpha$, say $-35^{\circ}$ & an above value of $\beta$, you can now vary $\gamma$ from $-180^{\circ} \text{ to }180^{\circ}$ to simulate the animation.
            - Now fix another $\alpha$, say $45^{\circ}$ and repeat the above process. So that's 2 specific values of $\alpha$.
            - Show this for all angles of $\beta$ if there are more than 1.
            - Therefore, when the code is run, there should be a minimum of (2 $\times$ (number of values of $\beta$)) animations. 2 for values of $\alpha$, you could show it for even more if you wish to.

In [47]:
##############################################################################
# TODO: Do tasks described in 2.4                                            #
##############################################################################
# Replace "pass" statement with your code
pass
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

> VOILA! You have just animated the famous Gimbal lock problem. If you are curious, read about the [Apollo 11](https://en.wikipedia.org/wiki/Gimbal_lock#On_Apollo_11) Gimbal lock incident.

### Questions for 2.4 (Optional)

- Mention the value(s) of $\beta$ here: 
    * Ans: *your answer here.*
- Now that you understand gimbal lock through visualization, explain it now in matrix form: For the above values of $\beta$, what does the rotation matrix look like? Can you explain why gimbal lock occurs from the rotation matrix alone? Clue: Use sin/cos formulae. 
    * Ans: *your answer here.*
- Call the function `two_four` for the visualization of gimbal lock written above. 

In [48]:
#uncomment the following and add input parameters if any
#two_four()